In [1]:
import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import date

In [2]:
from mod_my_utils.read_write_data import *

In [3]:
folder_path = 'data/'
stock_name = 'AAPL'
index_col = 'Date'

In [4]:
stock_data = get_price_csv(folder_name=folder_path,symbol=stock_name, index_col=index_col)

stock name:  AAPL


In [5]:
data = stock_data[["Adj. Close"]].copy()
data.columns = ["price"]
data.head()

,price
Date,
2007-12-03,22.986003
2007-12-04,23.108091
2007-12-05,23.839335
2007-12-06,24.411582
2007-12-07,24.970258


### ROC Strategy and portfolio

In [6]:
from mod_backtest_utils.backtest import Strategy, Portfolio

In [7]:
class ROCStrategy(Strategy):
    """
    Requires:
    symbol: A stock symbol on which to form a strategy
    bars: A DataFrame of bars for the above symbol
    period: Lookback period for roc calculation
    """
    
    def __init__(self, symbol, bars, period, buy_threshold, sell_threshold):
        self.symbol = symbol
        self.bars = bars
        self.period = period
        self.buy_threshold = buy_threshold
        self.sell_threshold = sell_threshold
 
    def generate_signals(self):
        """
        Returns the DataFrame of signal containing the signals to go long, short or hold (1, -1, 0)
        """
        # initiate signal data frame with date as index
        signals = pd.DataFrame(index=self.bars.index)
        signals['price'] = self.bars['price']
        
        # calculate roc values for the given period
        signals['roc' + str(self.period)] = signals['price'].diff(periods=self.period) / signals['price'].shift(periods=self.period)
        
        # buy sell signal generations
        signals['buy']= 0.0
        signals['sell']= 0.0
        signals['buy'] = np.where(signals['roc' + str(self.period)] < self.buy_threshold, -1.0, 0.0)
        signals['sell'] = np.where(signals['roc' + str(self.period)] > self.sell_threshold, 1.0, 0.0)
        signals['buy'] = signals['buy'].diff()
        signals['sell'] = signals['sell'].diff()
        signals.loc[signals['buy']==-1.0,['buy']] = 0.0 
        signals.loc[signals['sell']== 1.0,['sell']] = 0.0 
        signals['buy_sell'] = signals['buy'] + signals['sell']
        return signals[['price', 'buy_sell']]

In [10]:
symbol = 'AAPL'
buy_threshold = -0.2
sell_threshold = 0.2
period = 12
roc = ROCStrategy(symbol= symbol, bars= data, period= period, buy_threshold=buy_threshold, sell_threshold=sell_threshold)

In [11]:
roc.generate_signals()

,price,buy_sell
Date,,
2007-12-03,22.986003,NaN
2007-12-04,23.108091,0.0
2007-12-05,23.839335,0.0
2007-12-06,24.411582,0.0
2007-12-07,24.970258,0.0
...,...,...
2017-11-27,174.090000,0.0
2017-11-28,173.070000,0.0
2017-11-29,169.480000,0.0
